# GUI design tests

After adding a few buttons, it became clear that it's easy to write messy code for a GUI. A major difficulty are the numerous links between the graphical elements that contrasts to the desire to write reusable code. The default GUI elemenents in IPyWidgets are reusable, but they need call backs and update methods to function. It is possible o hardwire all this from the outside, but the number of links increases rapidly with the number of widgets on the screen.

## Centralized variables

By default, the variables represented eg by a slider are not linked to the other GUI elements. Below, we attempt an alternative approach where the widgets are linked by an object called a variable. The aim is to connect by default all elements called time for example. If a display needs two time variables, then these can be given different names.

In [1]:
# Imports

# arrays
import xarray as xr
import numpy as np 
import zarr
# plotting
import matplotlib.pyplot as plt 
%matplotlib widget
plt.rcParams['figure.figsize'] = [10, 5]
# gui elements
import ipywidgets as widgets
from ipyleaflet import *

# 

In [63]:
#define Variables

class Observer: #base class for observers
    def update(self,v: Variable):
        # you should override this function with something meaningful
        pass

class Variable:
    def __init__(self,name: str, label:str):
        self._name=name
        self._label=label
        self._observers=[]
        self._value=None
    def observe(self,observer:Observer): 
        # an observer is a function that is called when the variable is changed
        # the only argument is the variable
        self._observers.append(observer)
    @property
    def name(self): # to allow for temp=variable.name
        return self._name
    #make immutable
    #@name.setter 
    #def name(self,new_name: str): # allow for variale.name=new_name
    #    self._name=new_name
    @property
    def label(self):
        return self._label
    #@label.setter
    #def label(self,new_label: str):
    #    self._label=new_label

class IntVariable(Variable):
    def __init__(self,name: str,label: str, min_val:int, max_val:int, initial_val:int):
        super().__init__(name,label)
        self._min=min_val
        self._max=max_val
        self._value=initial_val
    @property
    def value(self):
        return self._value
    @value.setter
    def value(self,new_value: int):
        print(f"set value of {self.name} to {new_value}\n")
        #truncate to range
        v=max(new_value,self._min)
        v=min(v,self._max)
        self._value=v
        #inform observers
        for obs in self._observers:
            obs.update(self)
    @property
    def min(self):
        return self._min
    @property
    def max(self):
        return self._max


In [64]:
#demo of Variable class
v1=IntVariable("layer","Vertical layer number",1,10,1)
print(f'variable name={v1.name} label={v1.label}\n')
print(f'current {v1.name} has value {v1.value} of range {v1.min}:{v1.max}\n')

v1.value=3
print("change value\n")
print(f'current {v1.name} has value {v1.value} of range {v1.min}:{v1.max}\n')


variable name=layer label=Vertical layer number

current layer has value 1 of range 1:10

set value of layer to 3

change value

current layer has value 3 of range 1:10



In [82]:
class WtIntSlider(widgets.IntSlider):
    def __init__(self,v: IntVariable, orientation='horizontal'):
        super().__init__(value=v.value, min=v.min, max=v.max, step=1,
        description=v.label, disabled=False, continuous_update=False,
        orientation=orientation) #, readout=True, readout_format='d' 
        self._variable=v
        self.observe(self.update_from_slider,names="value")
        v.observe(self)
    def update_from_slider(self,obj):
        print("signal from slider")
        self._variable.value=self.value
    def update(self,v: IntVariable):
        print(f"slider receives signal from variable {v.name}. Value is {v.value}\n")
        self.value=str(v.value)

class WtIntLabel(widgets.Label):
    def __init__(self,v: IntVariable, orientation='horizontal'):
        super().__init__(value=str(v.value), description=v.label, 
        disabled=False, orientation=orientation)
        self._variable=v
        v.observe(self)
    def update(self,v: IntVariable):
        print(f"signal from variable {v.name}. Value is {v.value}\n")
        self.value=str(v.value)

In [83]:
# Demo for Int slider and label.
# The widgets are connected with the variable
v2=IntVariable("itime","Time-step",1,100,1)
sld2a=WtIntSlider(v2)
sld2b=WtIntSlider(v2)
lbl2=WtIntLabel(v2)
display(sld2a,sld2b,lbl2)


WtIntSlider(value=1, continuous_update=False, description='Time-step', min=1)

WtIntSlider(value=1, continuous_update=False, description='Time-step', min=1)

WtIntLabel(value='1', description='Time-step')

In [84]:
v2._observers, v2._value

([WtIntSlider(value=55, continuous_update=False, description='Time-step', min=1),
  WtIntSlider(value=55, continuous_update=False, description='Time-step', min=1),
  WtIntLabel(value='55', description='Time-step')],
 55)